# 06_saving_consecutive_metrics_all_models
* Calculating the consecutive metrics for all models and then saving this to a folder with in the consecutve metrics folder with the folder name being the window length.

PHD-10
* Newer more streamlined version from Documents/PhD/03_SN_global_different_windows_(nb25).ipynb
* ALl S/N calculation and bound are handled by function

# Preamble

In [35]:
import numpy as np
import os
from time import perf_counter
import itertools
import matplotlib.pyplot as plt
import xarray as xr
from importlib import reload
from glob import glob
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from enum import Enum
from dask.diagnostics import ProgressBar
from typing import Dict, List
from warnings import filterwarnings
filterwarnings('ignore')

In [123]:
import sys
sys.path.append(constants.MODULE_DIR)
import xarray_class_accessors as xca
import sn_plotting
import signal_to_noise
import open_ds
from classes import ExperimentTypes
import constants
import utils
from itertools import chain


logger = utils.get_notebook_logger()

In [36]:
chunks = {'lat':48/2,'lon':96/2,'time':-1}

In [32]:
RUN_PARAMETERS = {'window': 101, 'requested_length': 500}

In [161]:
variable = 'tas'
ROOT_DIR = os.path.join(constants.LONGRUNMIP_DIR, variable)
ROOT_DIR_REGRID = os.path.join(ROOT_DIR, 'regrid_retimestamped')
ROOT_DIR_LOWESS = os.path.join(ROOT_DIR, 'lowess')

ROOT_DIR_REGRID

'/g/data/w40/ab2313/PhD/longrunmip/tas/regrid_retimestamped'

In [162]:
models_to_get = open_ds.get_models_longer_than_length()
utils.pprint_list(models_to_get, num_start_items=7)

lenght = 7
 0. ccsm3
 1. cesm104
 2. cnrmcm61
 3. hadcm3l
 4. ipslcm5a
 5. mpiesm11
 6. mpiesm12


In [163]:
files_to_open_experiment = open_ds.get_file_names_from_from_directory(ROOT_DIR_REGRID,
                                                           ExperimentTypes.ABRUPT4X,
                                                           models_to_get)

files_to_open_control = open_ds.get_file_names_from_from_directory(ROOT_DIR_REGRID,
                                                           ExperimentTypes.CONTROL,
                                                           models_to_get)
utils.pprint_list(files_to_open_experiment)

lenght = 7
 0. tas_mon_CCSM3_abrupt4x_2120_g025.nc
 1. tas_mon_CESM104_abrupt4x_5900_g025.nc


In [164]:
models_control = [fname.split('_')[2].lower() for fname  in files_to_open_control]
models_experiment = [fname.split('_')[2].lower() for fname  in files_to_open_experiment]
models = np.intersect1d(models_control, models_experiment)
models

array(['ccsm3', 'cesm104', 'cnrmcm61', 'hadcm3l', 'ipslcm5a', 'mpiesm11',
       'mpiesm12'], dtype='<U8')

In [165]:
RUN_PARAMETERS = {'window': 51, 'requested_length': 1000}

In [166]:
OUTPUT_DIR = os.path.join(ROOT_DIR, 'signal_to_noise')
OUTPUT_FOLDER = 'window_{a}_length_{b}'.format(a=RUN_PARAMETERS['window'], 
                                               b=RUN_PARAMETERS['requested_length'])
OUTPUT_DIR_2 = os.path.join(OUTPUT_DIR, OUTPUT_FOLDER)
                            
print(OUTPUT_DIR, OUTPUT_DIR_2, sep='\n')

/g/data/w40/ab2313/PhD/longrunmip/tas/signal_to_noise
/g/data/w40/ab2313/PhD/longrunmip/tas/signal_to_noise/window_51_length_1000


In [167]:
try:
    os.mkdir(OUTPUT_DIR)
    os.mkdir(OUTPUT_DIR_2)
except FileExistsError as e:
    pass

In [168]:
def open_all_needed_netcdf_files(control_fpath, experiment_fpath):
    
    ds_experiment = open_ds.read_longrunmip_netcdf(experiment_fpath, ROOT_DIR_REGRID)#.to_array()
    ds_control = open_ds.read_longrunmip_netcdf(control_fpath, ROOT_DIR_REGRID)#.to_array()
    
    ds_experiment_lowess = open_ds.read_longrunmip_netcdf(experiment_fpath, ROOT_DIR_LOWESS)#.to_array()
    ds_control_lowess = open_ds.read_longrunmip_netcdf(control_fpath, ROOT_DIR_LOWESS)#.to_array()
    
    return ds_experiment, ds_control, ds_experiment_lowess, ds_control_lowess

In [169]:
def save_all_netcdf_files(ds_sn, ds_sn_stable, ds_sn_increasing, ds_sn_decreasing):
    ds_sn.to_netcdf(os.path.join(OUTPUT_DIR_2, f'{model}_signal_to_noise.nc'))
    ds_sn_stable.to_netcdf(os.path.join(OUTPUT_DIR_2, f'{model}_signal_to_noise_stable.nc'))
    ds_sn_increasing.to_netcdf(os.path.join(OUTPUT_DIR_2, f'{model}_signal_to_noise_increasing.nc'))
    ds_sn_decreasing.to_netcdf(os.path.join(OUTPUT_DIR_2, f'{model}_signal_to_noise_decreasing.nc'))

In [170]:
reload(open_ds)
for model in models[1:]:
    
    t1 = perf_counter()
    print(model, end = '')
    
    control_fpath = [fname for fname in files_to_open_control if model in fname.lower()][0]
    experiment_fpath = [fname for fname in files_to_open_experiment if model in fname.lower()][0]
    
    try:
        ds_experiment, ds_control, ds_experiment_lowess, ds_control_lowess =\
                    open_all_needed_netcdf_files(control_fpath, experiment_fpath)
        print('| All files have been opened')

        ds_sn, ds_sn_stable, ds_sn_increasing, ds_sn_decreasing =\
                signal_to_noise.global_sn(ds_experiment, ds_control, 
                             ds_experiment_lowess, ds_control_lowess, 
                             window=RUN_PARAMETERS['window'])

        save_all_netcdf_files(ds_sn, ds_sn_stable, ds_sn_increasing, ds_sn_decreasing)

        t2 = perf_counter()
        print(f' - {(t2-t1)/60}')
        
    except LongRunMIPError as e:
        logger.error(e)
    

cesm104| All files have been opened
 - 66.06457434973369
cnrmcm61| All files have been opened
 - 73.29777148726086
hadcm3l| All files have been opened
 - 63.7271091418496
ipslcm5a| All files have been opened
 - 63.00092467530631
mpiesm11| All files have been opened
 - 72.69829579664705
mpiesm12| All files have been opened
 - 65.3362397527943
